In [1]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.exa import ExaTools

In [2]:
md = "llama-3.3-70b-versatile"
workout_data = ""
reciepe_data= ""

In [3]:
with open('prev_data_workout.txt', 'r') as file:
    workout_data = file.read()

with open('prev_data_reciepe.txt', 'r') as file:
    reciepe_data = file.read()

In [4]:
cook_agent = Agent(
    description="You will help the user with reciepes with the ingredients they have.",
    model = Groq(id=md),
    name ="Chef",
    tools=[ExaTools(api_key= "de8ed4f8-7cd1-4be0-8e2b-7d9beeb7f21b")],
    add_history_to_messages=True,
    markdown=True,
    instructions=[
        "Search for recipes based on the ingredients and time available from the knowledge base.",
        "Include the exact calories, preparation time, cooking instructions, and highlight allergens for the recommended recipes.",
        "Always search exa for recipe links or tips related to the recipes apart from knowledge base.",
        "Provide a list of recipes that match the user's requirements and preferences.",
        "If the user has not provided many ingredients or only core element of the food, provide an easy reciepe which doesn't require many ingredients."
        "Always return steps of the cooking process"
    ],
)

In [5]:
exercise_agent = Agent(
    description="you will create for the user a workout plan based on their preferences.",
    model = Groq(id=md),
    name = "Trainer",
    instructions=[
        "You are a highly skilled fitness trainer agent",
        "Your goal is to create a workout plan for your users, and if they provide a workout plan you should be able to improve it",
        "The workout plan should be matched to the goal that the user wants to achieve",
        "Include for every exercise how it should be done and what muscles are activated during the same.",
        "The size of output should be less than 6000 words"
    ]
    , 
    tools=[ExaTools(api_key= "de8ed4f8-7cd1-4be0-8e2b-7d9beeb7f21b")]
)

In [16]:
summary_agent = Agent(
    description="Your job is to summarise the given workout plan or receipe by extracting key elements.",
    model = Groq(id=md),
    name ="Summarizer",
    instructions=[
        "You are an expert summarizer and data analyser",
        "Your goal is to analyse the input and extract key highlights from the input",
        "If input is workout plan, identify the goals the workout is planning to achieve, the strain it puts on user and more valuable insights that will help another agent to create a receipe",
        "If the input is a reciepe, ensure that you are taking the key elements and macros they are providing and return key inferences you make. Ensure that the ouput can be used for creating a workout plan",
    ],
    tools=[ExaTools(api_key= "de8ed4f8-7cd1-4be0-8e2b-7d9beeb7f21b")]
    

)

In [ ]:
while True:
    print("About the user: ")
    print("\n Hi User! What Can I help with you today?")
    print("\n 1. Exercise Agent")
    print("\n 2. Cook me something")
    print(" What's your mood today : ")
    choice= int(input())

    if choice==1:
        age = input("Enter Age : ")
        sex = input("Enter Sex : ")
        exercise_freq = input("Enter current exe freq: ")
        goal = input("Enter Goal: ")
        days_to_workout = input("No. of days to workout per week: ")
        gym_access = "I have access to Gym" if input("Do you have access to gym: ")=="Yes" else "I do not have access to Gym"
        include_steps ="Include cardio and steps goal" if input("Tailor workout to include steps goal? ")=="Yes" else ""

        input_statement = f"""
        Hi trainer!! My name is User. I am a {age} year old {sex} who exercises {exercise_freq}. 
        My goal is to {goal}. Please provide me with a workout plan for {days_to_workout} days per week. {gym_access}. 
        {include_steps}
"""
        print(input_statement)
        response = exercise_agent.run(input_statement, stream=False)
        print(response)

        while True:
            if input("Do you want to make any changes to the output?(Y/N) : ") == "Y":
                changes = input("Enter Changes : ")
                input_statement = input_statement + f"Plase conside the following points as important : {changes}"
                response = ""
                for delta in exercise_agent.run(input_statement):
                    response += delta
            else:
                break

        save= input("Do you want to save the workout plan? (Y?N ): ")
        if save =="Y":
            with open('prev_data_workout.txt', 'a') as file:
                file.write(response)

    elif choice ==2:
        cook_agent.print_response(
            'I have chicken with me. Suggest me 3 good and quick reciepes for it which are preferrably a balanced meal',stream = True
        )

    else:
        print("Please select a valid option")

In [6]:
age = input("Enter Age : ")
sex = input("Enter Sex : ")
exercise_freq = input("Enter current exe freq: ")
goal = input("Enter Goal: ")
days_to_workout = input("No. of days to workout per week: ")
gym_access = "I have access to Gym" if input("Do you have access to gym: ")=="Yes" else "I do not have access to Gym"
include_steps ="Include cardio and steps goal" if input("Tailor workout to include steps goal? ")=="Yes" else ""
input_statement = f"""
        Hi trainer!! My name is User. I am a {age} year old {sex} who exercises {exercise_freq}. 
        My goal is to {goal}. Please provide me with a workout plan for {days_to_workout} days per week. {gym_access}. 
        {include_steps}
"""
print(input_statement)
response = exercise_agent.run(input_statement, stream=False)


        Hi trainer!! My name is User. I am a 23 year old Male who exercises daily. 
        My goal is to Muscular body. Please provide me with a workout plan for 6 days per week. I have access to Gym. 
        Include cardio and steps goal



In [7]:
print(response)

content="As your trainer, I'll provide you with a 6-day workout plan to help you achieve your goal of a muscular body. Since you have access to a gym, we'll incorporate a mix of compound exercises and isolation exercises to target all major muscle groups.\n\n**Day 1: Chest and Triceps**\n\n1. Warm-up: 5-10 minutes of cardio (treadmill, bike, or elliptical) and dynamic stretching (arm circles, leg swings, etc.)\n2. Barbell Bench Press: 4 sets of 8-12 reps\n\t* Lie on a flat bench and grip the barbell with your hands slightly wider than shoulder-width apart\n\t* Lower the bar to your chest, then press it upwards, extending your arms fully\n\t* Muscles activated: Pectoralis major, Anterior deltoids, Triceps brachii\n3. Incline Dumbbell Press: 3 sets of 10-15 reps\n\t* Sit on an incline bench and hold a dumbbell in each hand\n\t* Press the dumbbells upwards, extending your arms fully, then lower them back to the starting position\n\t* Muscles activated: Upper Pectoralis major, Anterior del

In [9]:
response.content

"As your trainer, I'll provide you with a 6-day workout plan to help you achieve your goal of a muscular body. Since you have access to a gym, we'll incorporate a mix of compound exercises and isolation exercises to target all major muscle groups.\n\n**Day 1: Chest and Triceps**\n\n1. Warm-up: 5-10 minutes of cardio (treadmill, bike, or elliptical) and dynamic stretching (arm circles, leg swings, etc.)\n2. Barbell Bench Press: 4 sets of 8-12 reps\n\t* Lie on a flat bench and grip the barbell with your hands slightly wider than shoulder-width apart\n\t* Lower the bar to your chest, then press it upwards, extending your arms fully\n\t* Muscles activated: Pectoralis major, Anterior deltoids, Triceps brachii\n3. Incline Dumbbell Press: 3 sets of 10-15 reps\n\t* Sit on an incline bench and hold a dumbbell in each hand\n\t* Press the dumbbells upwards, extending your arms fully, then lower them back to the starting position\n\t* Muscles activated: Upper Pectoralis major, Anterior deltoids\n4

In [10]:
print(str(response.content))

As your trainer, I'll provide you with a 6-day workout plan to help you achieve your goal of a muscular body. Since you have access to a gym, we'll incorporate a mix of compound exercises and isolation exercises to target all major muscle groups.

**Day 1: Chest and Triceps**

1. Warm-up: 5-10 minutes of cardio (treadmill, bike, or elliptical) and dynamic stretching (arm circles, leg swings, etc.)
2. Barbell Bench Press: 4 sets of 8-12 reps
	* Lie on a flat bench and grip the barbell with your hands slightly wider than shoulder-width apart
	* Lower the bar to your chest, then press it upwards, extending your arms fully
	* Muscles activated: Pectoralis major, Anterior deltoids, Triceps brachii
3. Incline Dumbbell Press: 3 sets of 10-15 reps
	* Sit on an incline bench and hold a dumbbell in each hand
	* Press the dumbbells upwards, extending your arms fully, then lower them back to the starting position
	* Muscles activated: Upper Pectoralis major, Anterior deltoids
4. Cable Fly: 3 sets 

In [2]:
shopping_dict = {
    "Trader Joe's": [
        ("Chicken Breast (2 lbs)", 7.40),
        ("Tofu (1 lb)", 2.29),
        ("Spinach (1 lb)", 3.49),
        ("Tomatoes (2 lbs)", 3.79),
        ("Onions (2 lbs)", 1.99),
        ("Garlic (1 Bulb)", 0.99),
        ("Ginger (0.5 lb)", 2.49),
        ("Bok Choy (1 lb)", 2.99),
        ("Brown Rice (1 lb)", 1.99),
        ("Chili Paste (1 Jar)", 2.99),
        ("Avocado (2)", 2.58),
        ("Bananas (2 lbs)", 1.18)
    ],
    "Ralphs": [
        ("Eggs (1 Dozen)", 3.00),
        ("Paneer (Low-Fat) (1 lb)", 7.00),
        ("Cauliflower (1 Head)", 4.00),
        ("Green Chilies (0.25 lb)", 1.50),
        ("Broccoli (1 Head)", 3.50),
        ("Bell Peppers (2)", 3.00),
        ("Carrots (1 lb)", 1.50),
        ("Green Beans (1 lb)", 3.50),
        ("Soy Sauce (Low Sodium) (1 Bottle)", 4.00),
        ("Apples (2 lbs)", 4.00),
        ("Cilantro", 0.99),
        ("Yogurt (Large Tub)", 5.00)
    ],
    "Target": [
        ("Whole Wheat Noodles (1 lb)", 2.75),
        ("Lentils (Toor/Masoor) (1 lb)", 3.25),
        ("Rice Vinegar (1 Bottle)", 3.75),
        ("Sesame Oil (1 Bottle)", 7.50),
        ("Almonds (1 lb)", 7.99),
        ("Protein Powder", 20.00)
    ]
}

In [10]:
[sum(price for item, price in shopping_dict[i]) for i in shopping_dict]

[34.169999999999995, 40.99, 45.24]

In [6]:
for i in shopping_dict:
    total = 0
    items = []
    for j in shopping_dict[i]:
        items.append([j[0],j[1]])
        total = total+j[1]
    print(i+" cost: $"+ str(total))
    print(items)

Trader Joe's cost: $34.169999999999995
[['Chicken Breast (2 lbs)', 7.4], ['Tofu (1 lb)', 2.29], ['Spinach (1 lb)', 3.49], ['Tomatoes (2 lbs)', 3.79], ['Onions (2 lbs)', 1.99], ['Garlic (1 Bulb)', 0.99], ['Ginger (0.5 lb)', 2.49], ['Bok Choy (1 lb)', 2.99], ['Brown Rice (1 lb)', 1.99], ['Chili Paste (1 Jar)', 2.99], ['Avocado (2)', 2.58], ['Bananas (2 lbs)', 1.18]]
Ralphs cost: $40.99
[['Eggs (1 Dozen)', 3.0], ['Paneer (Low-Fat) (1 lb)', 7.0], ['Cauliflower (1 Head)', 4.0], ['Green Chilies (0.25 lb)', 1.5], ['Broccoli (1 Head)', 3.5], ['Bell Peppers (2)', 3.0], ['Carrots (1 lb)', 1.5], ['Green Beans (1 lb)', 3.5], ['Soy Sauce (Low Sodium) (1 Bottle)', 4.0], ['Apples (2 lbs)', 4.0], ['Cilantro', 0.99], ['Yogurt (Large Tub)', 5.0]]
Target cost: $45.24
[['Whole Wheat Noodles (1 lb)', 2.75], ['Lentils (Toor/Masoor) (1 lb)', 3.25], ['Rice Vinegar (1 Bottle)', 3.75], ['Sesame Oil (1 Bottle)', 7.5], ['Almonds (1 lb)', 7.99], ['Protein Powder', 20.0]]


In [ ]:

grocery_data = {
    'Trader Joe\'s': [
        ('Brown Rice (2 lbs)', 6.00),
        ('Whole Wheat Pasta (1 lb)', 1.75),
        ('Lentils (Toor & Masoor Dal, 1 lb each)', 5.00),
        ('Spinach (1 lb)', 3.00),
        ('Cauliflower (1 head)', 3.00),
        ('Tomatoes (3 lbs)', 13.50),
        ('Onions (2 lbs)', 4.00),
        ('Garlic (1 head)', 1.50),
        ('Ginger (0.25 lb)', 2.00),
        ('Green Chilies (0.25 lb)', 1.50),
        ('Low-fat Yogurt (32 oz)', 3.00)
    ],
    'Target': [
        ('Whole Wheat Flour (5 lbs)', 5.50),
        ('Chicken Breast (4 lbs)', 14.00),
        ('Eggs (24)', 7.00),
        ('Broccoli (1 head)', 2.50),
        ('Zucchini (1 lb)', 2.50),
        ('Bell Peppers (2)', 3.50),
        ('Cumin (2 oz)', 3.50),
        ('Coriander Powder (2 oz)', 2.75),
        ('Turmeric (2 oz)', 3.50),
        ('Chili Powder (2 oz)', 2.75)
    ],
    'Ralphs': [
        ('Paneer (low-fat, 1 lb)', 9.00),
        ('Parmesan Cheese (4 oz)', 5.00),
        ('Ricotta Cheese (15 oz)', 6.00),
        ('Tomato Paste (6 oz)', 2.00),
        ('Garam Masala (2 oz)', 5.00),
        ('Mustard Seeds (2 oz)', 2.00),
        ('Curry Leaves (1 bunch)', 2.50)
    ]
}


SyntaxError: invalid syntax (3832407145.py, line 1)

In [2]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.model.google import Gemini
from phi.tools.exa import ExaTools
def getter(prompt):
    with open("small_database.txt","r") as file:
        small_db = file.read()

    md = Gemini(id="gemini-2.0-flash")
    # md = Groq(id="deepseek-r1-distill-llama-70b")
    grocery_list_agent =Agent(
        description="You are a professional shopper and chef. You will help user with creating a grocery list to buy items from specified stores and fit within the given budget.",
        model = md,
        name = "Shopper",
        instructions=[
        "You will create a grocery list from specified ingredients and their quantities and a prompt as an input.",  
    "Retrieve ingredient information and prices from Ralphs, Target, and Trader Joe’s.",  
    "Provide prices for each ingredient from all three stores.",  
    """Present the results in a clear, tabulated format with columns for:  
        - Ingredient,  
        - Quantity,  
        - Price at Ralphs,  
        - Price at Target,  
        - Price at Trader Joe’s,  
        - Total Cost (Including Quantity).""",  
    "You are allowed to add or remove ingredients from the given list if necessary for availability or cost-effectiveness.",  
    "Provide a total estimated cost for all ingredients, considering the specified quantities.",  
    "Ensure the output remains concise and clear, with a maximum length of 6000 words.",  
    "Ensure the final output includes the store-specific prices and the total cost reflecting the required quantities.",  
    "The output should be in a clean list format for easy reference.",  
    "Use the following information for price reference ( this is real time data)",
    small_db,"If you can't find the required prices here, use the exa toll provided."

        ]
        
    )

    # prompt= f"""
    # I need a grocery list for 4 weeks, including vegetables, chicken and eggs. 
    # I want to have lunch, dinner and breakfast every day, with each meal serving 1 people. 
    # I have nothing in my pantry. 
    # I prefer {cuisine_ingredients['Indian']}, {cuisine_ingredients['Italian']} recipes but am open to others if necessary. 
    # Provide the quantity required for each ingredient as well.
    # Adjust the ingredient quantities based on my preferences and should last for the provided duration.
    # I want a balanced meal plan and donot mind some leftovers.
    # create me a grocery list to buy from ralphs, target and trader joes. 
    # The list should have the prices mentioned for the store and approximate budget of $120.
    # Make sure the total cost is as close to the budget. 
    # only output the ingredients and their quantity required
    # """

    # cost_estimator = Agent(
    #     description="You are a grocery list cost estimator",
    #     model = md,
    #     name = "Cost estimator",
    #     instructions=[
    #             "You will receive a grocery shopping list as input.",  
    # "Create a to-do list along with the prices for the items.",  
    # "For each ingredient, provide the prices from Ralphs, Target, and Trader Joe’s.",  
    # "Add the total price at the end, reflecting the best choices for each store.",  
    # "Ensure you select the best price for each ingredient across all three stores.",  
    # "Distribute the items among different stores instead of purchasing everything from one store.",  
    # "Make sure the total cost is within $15 of the $120 budget (between $105 and $135).",  
    # "Adjust the list by adding or removing items to meet the budget as accurately as possible.",  
    # "Present the results in a clear and organized format, divided by stores indicating where each item should be bought.",  
    #     ]
    #     , 
    #     tools=[ExaTools(api_key= "de8ed4f8-7cd1-4be0-8e2b-7d9beeb7f21b")]
    #     )
    

    to_creator = Agent(
        description="You are a to do list generatot",
        model = md,
        name = "List generator",
        instructions=[
        "You will receive a grocery list with specified ingredients and their quantities as input.",  
"Generate a grocery list for each store (Trader Joe's, Target, and Ralphs).",  
"Organize the output into a dictionary format with the store names as keys and the items as lists of tuples.",  
"If duration is more than 2 weeks, divide the grocery list such that the ingredients are bought on a 2 week basis to ensure freshness else do not divide. The output in case of less than 2 weeks should have First week mentioned in the final dict",
"Each tuple should contain the item name (as a string) and the price (as a float, reflecting the cost per specified quantity).",  
"Present the output in the following format always!:",  
"grocery_data = {",
    "First week:",
"    'Trader Joe\'s': [",  
"        ('Chicken Breast (3 lbs)', 21.00,Week1) , ",  
"        ('Eggs (24)', 5.00),",  
"        ('Lentils (Toor/Masoor) (2 lbs)', 6.00),",  
"        # More items",  
"    ],",  
"    'Target': [",  
"        ('Paneer (Low-Fat) (1 lb)', 7.00),",  
"        ('Bell Peppers (3)', 4.50),",  
"        # More items",  
"    ],",  
"    'Ralphs': [",  
"        ('Avocado (2)', 4.00),",  
"        # More items",  
"    ]",  
"}",  
"For each ingredient, find the best price from the available stores and choose it.",  
"Distribute items across the stores, ensuring that not all items are bought from a single store.",  
"Ensure the list is balanced and doesn’t exceed the expected budget for each store. Make this top priority It is okay to be + or - $15 of the total value",  
"Maintain the specified quantities and ingredients consistently across all stores.",  
"Output the list in the exact format shown above with each item and its cost correctly aligned.",


        ]
    )

    response = grocery_list_agent.run(prompt, stream=False)
        
        # prompt = response.content + " Use the following list to estimate the cost of items from provided stores"
        # response = cost_estimator.run(prompt,stream=False)
        
    prompt = response.content + " Create a to-do list for the list of grocery items to buy."
    response2 = to_creator.run(prompt,stream=False)
     

    return str(response2.content)


/Users/aryanparab/Desktop/Practice/node-course/Untitled Folder/Agentic/venv1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
p1 ="""I need a grocery list for 4 weeks, including ['Eggs', 'Vegetables']. 
    I want to have ['Lunch'] every day, with each meal serving 1 people. 
    I have  in my pantry. 
    I prefer the following cusines: Rice Whole Wheat Flour (for Roti) Lentils (Toor Dal, Masoor Dal) Chicken Breast Eggs Paneer (low-fat) Spinach Cauliflower Tomatoes Onions Garlic Ginger Green Chilies Cumin Coriander Powder Turmeric Garam Masala Chili Powder Mustard Seeds Curry Leaves Olive Oil Low-fat Yogurt (optional)Chicken Breast Ground Turkey (lean) Eggs Brown Rice Whole Wheat Pasta Sweet Potatoes Spinach Kale Bell Peppers Carrots Broccoli Olive Oil Garlic Onions Black Pepper Paprika Oregano Basil Greek Yogurt (low-fat) Quinoa Almonds AvocadoBrown Rice Whole Wheat Tortillas Chicken Breast Eggs Black Beans Pinto Beans Tomatoes Corn Avocado Bell Peppers Onions Garlic Cilantro Lime Cumin Chili Powder Oregano Jalapeños Olive Oil Low-fat Greek Yogurt (as sour cream alternative).
    Provide a grocery list in the budget of $120
    Adjust the ingredient quantities based on my preferences and should last for the provided duration.
    I want a balanced meal plan and donot mind some leftovers."""

In [4]:
import ast
while True:

    try:
        st = getter(p1)
        d = ast.literal_eval(st.split('=')[1].strip().split("print")[0].strip().split("`")[0].strip())
        if d: 
            break
    except:
        pass

In [10]:
weeks = list(d.keys())
print(weeks)

['First week', 'Second week', 'Third week', 'Fourth week']


In [27]:
week_totals = []
target_list = []
ralph_list =[]
tj_list = []
for value in weeks:
    stores = d[value]
    week_tots = 0
    print(value)
    for store in stores:
        tots = sum([i[1] for i in stores[store]])
        print("\n",store," : ", tots,"\n")
        for product in stores[store]:
            print(product[0]," : ", product[1])
        if store == "Target":
            target_list.append(tots)
        if store == "Ralphs":
            ralph_list.append(tots)
        if store == "Trader Joe's":
            tj_list.append(tots)
        week_tots = week_tots + tots
    print("\n")
    week_totals.append(week_tots)

    print(sum(week_totals))

First week

 Trader Joe's  :  9.75 

Basmati Rice (1 lb)  :  2.25
Eggs (9)  :  2.62
Turmeric Powder (1 oz)  :  0.88
Olive Oil (13 fl oz)  :  2.5
Spinach (0.5 - 10oz bag)  :  1.5

 Target  :  6.43 

Whole Wheat Flour (1.25 lb)  :  0.87
Black Beans (0.5 lb)  :  1.25
Pinto Beans (0.5 lb)  :  0.75
Cumin Seeds (0.24 oz)  :  1.13
Coriander Powder (1.75 oz)  :  0.63
Garam Masala (0.25 ct)  :  0.8
Avocado (1)  :  1.0

 Ralphs  :  19.810000000000002 

Toor Dal (0.5 lb)  :  1.87
Masoor Dal (0.25 lb)  :  1.07
Chicken Breast (1 lb)  :  3.75
Tomatoes (1 lb)  :  2.0
Onions (0.75 lb)  :  1.5
Garlic (0.5 head)  :  0.75
Ginger (0.25 lb)  :  1.0
Green Chilies (1 oz)  :  0.75
Chili Powder (1.75 oz)  :  0.75
Mustard Seeds (1.75 oz)  :  0.75
Curry Leaves (0.32 oz)  :  1.87
Low-fat Yogurt (8 oz)  :  1.0
Carrots (0.5 lb)  :  1.0
Cauliflower (0.25 head)  :  1.25
Lime(1)  :  0.5


35.99
Second week

 Trader Joe's  :  7.25 

Basmati Rice (1 lb)  :  2.25
Eggs (9)  :  2.62
Turmeric Powder (1 oz)  :  0.88
Spinach 

[9.75, 7.25, 7.25, 7.25]

In [70]:
totals = {}
for i, data in d.items():
    weekly = {}
    for store, products in data.items():
        s = [i[1] for i in products]
        weekly[store] = sum(s)
    totals[i] = weekly

In [72]:
final_sum = [list(i.items()) for i in totals.values()]

In [2]:
d =[{'_id': ('67df953799f09a49c9678dee'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 21:59:34 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (36 ct)', 10.47], ['Garlic (3 bulbs)', 3.98], ['Olive Oil (50 fl oz)', 10.99], ['Brown Rice (4 lb)', 8.98]], 'Target': [['Black Beans (2 lb)', 5.16], ['Brown Rice (4 lb)', 7.98]], 'Ralphs': [['Pinto Beans (2 lb)', 5.47], ['Avocado (6 ct)', 6.0], ['Corn (6 ears)', 5.94]]}, 'Second week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []}, 'Third week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []}}}, {'_id': ('67df95e899f09a49c9678df0'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 22:02:32 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (12)', 3.49], ['Olive Oil (50 fl oz)', 10.99], ['Garlic (1 head)', 0.99], ['Basil (1 bunch)', 2.99]], 'Target': [['Brown Rice (2 lb)', 3.99], ['Coconut Milk (1 can - 13.5oz)', 2.1], ['Bell Peppers (2)', 3.0], ['Soy Sauce (10 fl oz)', 3.5]], 'Ralphs': [['Ginger (0.5 lb)', 0.75], ['Limes (2)', 1.0], ['Lemongrass (1.765oz)', 2.0], ['Cilantro (0.5 bunch)', 1.0], ['Spinach (0.5 lb)', 2.5]]}, 'Second week': {"Trader Joe's": [['Whole Wheat Pasta (1 lb)', 1.5], ['Rice Noodles (0.5 lb)', 2.0], ['Tofu (0.5 lb)', 2.5], ['Peanuts (12oz)', 3.5]], 'Target': [['Tomatoes (1.5 lb)', 3.0], ['Zucchini (1 lb)', 1.5], ['Parmesan Cheese (2 oz)', 4.0]], 'Ralphs': [['Onions (1.5 lb)', 1.5], ['Carrots (1 lb)', 1.0], ['Green Onions (0.5 bunch)', 1.0], ['Broccoli (1 head)', 1.5]]}, 'Third week': {"Trader Joe's": [['Eggs (12)', 3.49], ['Garlic (1 head)', 0.99], ['Basil (1 bunch)', 2.99]], 'Target': [['Brown Rice (1 lb)', 2.0], ['Coconut Milk (1 can - 13.5oz)', 2.1], ['Bell Peppers (1)', 1.5]], 'Ralphs': [['Ginger (0.5 lb)', 0.75], ['Limes (2)', 1.0], ['Lemongrass (1.765oz)', 2.0], ['Cilantro (0.5 bunch)', 1.0], ['Spinach (0.5 lb)', 2.5]]}, 'Fourth week': {"Trader Joe's": [['Whole Wheat Pasta (1 lb)', 1.5], ['Rice Noodles (0.5 lb)', 2.0], ['Tofu (0.5 lb)', 2.5], ['Peanuts (12oz)', 3.5]], 'Target': [['Tomatoes (1.5 lb)', 3.0], ['Zucchini (1 lb)', 1.5], ['Parmesan Cheese (2 oz)', 4.0]], 'Ralphs': [['Onions (1.5 lb)', 1.5], ['Carrots (1 lb)', 1.0], ['Green Onions (0.5 bunch)', 1.0], ['Broccoli (1 head)', 1.5]]}}}, {'_id': ('67df96bc99f09a49c9678df2'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 22:06:04 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (12 ct)', 3.49], ['Garlic (4 bulbs)', 1.99], ['Fish Sauce (1 bottle)', 4.99], ['Frozen Fish (1 lb)', 20.0], ['Brown Rice (2 lb)', 4.49]], 'Target': [['Coconut Milk (Light) (1 can)', 1.7]], 'Ralphs': [['Ginger (0.25 lb)', 0.75]]}, 'Second week': {"Trader Joe's": [['Eggs (12 ct)', 3.49], ['Brown Rice (2 lb)', 4.49], ['Frozen Fish (1 lb)', 20.0]], 'Target': [['Coconut Milk (Light) (1 can)', 1.7]], 'Ralphs': [['Ginger (0.25 lb)', 0.75]]}}, 'prompt': {'duration': 4, 'budget': '140', 'cuisines': ['Italian', 'Thai'], 'diet': ['Vegetables', 'Fish', 'Chicken'], 'servings': 1, 'meals': ['Lunch', 'Breakfast', 'Dinner'], 'comments': '', 'already_have': ''}}]
[{'_id': ('67df953799f09a49c9678dee'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 21:59:34 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (36 ct)', 10.47], ['Garlic (3 bulbs)', 3.98], ['Olive Oil (50 fl oz)', 10.99], ['Brown Rice (4 lb)', 8.98]], 'Target': [['Black Beans (2 lb)', 5.16], ['Brown Rice (4 lb)', 7.98]], 'Ralphs': [['Pinto Beans (2 lb)', 5.47], ['Avocado (6 ct)', 6.0], ['Corn (6 ears)', 5.94]]}, 'Second week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []}, 'Third week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []}}}, {'_id': ('67df95e899f09a49c9678df0'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 22:02:32 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (12)', 3.49], ['Olive Oil (50 fl oz)', 10.99], ['Garlic (1 head)', 0.99], ['Basil (1 bunch)', 2.99]], 'Target': [['Brown Rice (2 lb)', 3.99], ['Coconut Milk (1 can - 13.5oz)', 2.1], ['Bell Peppers (2)', 3.0], ['Soy Sauce (10 fl oz)', 3.5]], 'Ralphs': [['Ginger (0.5 lb)', 0.75], ['Limes (2)', 1.0], ['Lemongrass (1.765oz)', 2.0], ['Cilantro (0.5 bunch)', 1.0], ['Spinach (0.5 lb)', 2.5]]}, 'Second week': {"Trader Joe's": [['Whole Wheat Pasta (1 lb)', 1.5], ['Rice Noodles (0.5 lb)', 2.0], ['Tofu (0.5 lb)', 2.5], ['Peanuts (12oz)', 3.5]], 'Target': [['Tomatoes (1.5 lb)', 3.0], ['Zucchini (1 lb)', 1.5], ['Parmesan Cheese (2 oz)', 4.0]], 'Ralphs': [['Onions (1.5 lb)', 1.5], ['Carrots (1 lb)', 1.0], ['Green Onions (0.5 bunch)', 1.0], ['Broccoli (1 head)', 1.5]]}, 'Third week': {"Trader Joe's": [['Eggs (12)', 3.49], ['Garlic (1 head)', 0.99], ['Basil (1 bunch)', 2.99]], 'Target': [['Brown Rice (1 lb)', 2.0], ['Coconut Milk (1 can - 13.5oz)', 2.1], ['Bell Peppers (1)', 1.5]], 'Ralphs': [['Ginger (0.5 lb)', 0.75], ['Limes (2)', 1.0], ['Lemongrass (1.765oz)', 2.0], ['Cilantro (0.5 bunch)', 1.0], ['Spinach (0.5 lb)', 2.5]]}, 'Fourth week': {"Trader Joe's": [['Whole Wheat Pasta (1 lb)', 1.5], ['Rice Noodles (0.5 lb)', 2.0], ['Tofu (0.5 lb)', 2.5], ['Peanuts (12oz)', 3.5]], 'Target': [['Tomatoes (1.5 lb)', 3.0], ['Zucchini (1 lb)', 1.5], ['Parmesan Cheese (2 oz)', 4.0]], 'Ralphs': [['Onions (1.5 lb)', 1.5], ['Carrots (1 lb)', 1.0], ['Green Onions (0.5 bunch)', 1.0], ['Broccoli (1 head)', 1.5]]}}}, {'_id': ('67df96bc99f09a49c9678df2'), 'email': 'parab.aryan@gmail.com', 'time': 'Sat Mar 22 22:06:04 2025', 'grocery_list': {'First week': {"Trader Joe's": [['Eggs (12 ct)', 3.49], ['Garlic (4 bulbs)', 1.99], ['Fish Sauce (1 bottle)', 4.99], ['Frozen Fish (1 lb)', 20.0], ['Brown Rice (2 lb)', 4.49]], 'Target': [['Coconut Milk (Light) (1 can)', 1.7]], 'Ralphs': [['Ginger (0.25 lb)', 0.75]]}, 'Second week': {"Trader Joe's": [['Eggs (12 ct)', 3.49], ['Brown Rice (2 lb)', 4.49], ['Frozen Fish (1 lb)', 20.0]], 'Target': [['Coconut Milk (Light) (1 can)', 1.7]], 'Ralphs': [['Ginger (0.25 lb)', 0.75]]}}, 'prompt': {'duration': 4, 'budget': '140', 'cuisines': ['Italian', 'Thai'], 'diet': ['Vegetables', 'Fish', 'Chicken'], 'servings': 1, 'meals': ['Lunch', 'Breakfast', 'Dinner'], 'comments': '', 'already_have': ''}}]


[{'_id': '67df953799f09a49c9678dee',
  'email': 'parab.aryan@gmail.com',
  'time': 'Sat Mar 22 21:59:34 2025',
  'grocery_list': {'First week': {"Trader Joe's": [['Eggs (36 ct)', 10.47],
     ['Garlic (3 bulbs)', 3.98],
     ['Olive Oil (50 fl oz)', 10.99],
     ['Brown Rice (4 lb)', 8.98]],
    'Target': [['Black Beans (2 lb)', 5.16], ['Brown Rice (4 lb)', 7.98]],
    'Ralphs': [['Pinto Beans (2 lb)', 5.47],
     ['Avocado (6 ct)', 6.0],
     ['Corn (6 ears)', 5.94]]},
   'Second week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []},
   'Third week': {"Trader Joe's": [], 'Target': [], 'Ralphs': []}}},
 {'_id': '67df95e899f09a49c9678df0',
  'email': 'parab.aryan@gmail.com',
  'time': 'Sat Mar 22 22:02:32 2025',
  'grocery_list': {'First week': {"Trader Joe's": [['Eggs (12)', 3.49],
     ['Olive Oil (50 fl oz)', 10.99],
     ['Garlic (1 head)', 0.99],
     ['Basil (1 bunch)', 2.99]],
    'Target': [['Brown Rice (2 lb)', 3.99],
     ['Coconut Milk (1 can - 13.5oz)', 2.1],
     ['Bell P

In [6]:
from datetime import datetime
def convert(date_time):
    format = '%a %b %d %H:%M:%S %Y'
    datetime_str = datetime.strptime(date_time, format)
    return datetime_str

In [17]:
dts =  sorted([[i,convert(i['time'])] for i in d],key=lambda x: x[1],reverse=True)


In [ ]:
to_show = dts[0]

In [33]:
to_show = [ i[0] for i in dts if i[0]['time']=="Sat Mar 22 22:06:04 2025"]

In [36]:
to_show = [{'_id': ('67e063dcb7315b58ceabdcfb'), 'email': 'parab.aryan@gmail.com', 'recipe': "Okay, Chef! Here's a 3-week Mediterranean/American fusion meal plan using your grocery list. I've focused on using all ingredients efficiently and providing variety while sticking to your specified meals (Breakfast and Dinner) and serving sizes (1).\n\n**Important Notes:**\n\n*   **Seasoning:** I'm assuming you have basic seasonings like salt, pepper, and maybe some dried herbs (oregano, basil, thyme). Adjust seasoning to your liking.\n*   **Flexibility:** Feel free to swap meals between days if something sounds better to you on a different day. This is a guide, not a rigid rulebook!\n*   **Storage:** Store leftovers properly in the refrigerator.\n*   **Ingredient Optimization:** I've planned to use ingredients across multiple weeks to minimize waste.\n\n## **Week 1: Mediterranean Start**\n\n**Day 1: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with diced Tomatoes and Spinach (from Ralphs).\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Scramble in a pan.\n*   **Dinner:** Mediterranean Lentil Soup\n    *   *Ingredients:* 1/4 cup Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), 1/4 can Chickpeas, 1/2 Tomato (diced), water/broth, Lemon juice, Olive Oil.\n    *   *Instructions:* Saute onion and garlic in olive oil. Add lentils, tomatoes, chickpeas, and water/broth. Simmer until lentils are tender. Season with lemon juice, salt, and pepper.\n\n**Day 2: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Lamb and Rice Bowl with Cucumber and Tomato Salad\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced), Red Onion (thinly sliced).\n    *   *Instructions:* Combine rice, browned lamb, cucumber, tomato, and red onion.\n\n**Day 3: Wednesday**\n\n*   **Breakfast:** Omelette with Spinach and Tomato\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Cook as an omelette.\n*   **Dinner:** Chickpea and Vegetable Salad with Lemon Vinaigrette\n    *   *Ingredients:* 1/4 can Chickpeas, Lettuce, 1/4 Tomato (diced), 1/4 Cucumber (diced), Red Onion (thinly sliced), Lemon juice, Olive Oil.\n    *   *Instructions:* Combine all ingredients. Dress with lemon juice and olive oil.\n\n**Day 4: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Olives and Red Onion\n    *   *Ingredients:* 2 Eggs, 1 tbsp Olives (chopped), 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add olives and red onion. Scramble in a pan.\n*   **Dinner:** Lamb and Lentil Stew\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/4 cup Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), water/broth, Tomato (diced).\n    *   *Instructions:* Saute onion and garlic in olive oil. Add lamb, lentils, tomatoes, and water/broth. Simmer until lentils are tender.\n\n**Day 5: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Greek Salad with Chickpeas\n    *   *Ingredients:* Lettuce, Tomato, Cucumber, Red Onion, 1/4 can Chickpeas, Olives, Lemon juice, Olive Oil.\n    *   *Instructions:* Combine all ingredients. Dress with lemon juice and olive oil.\n\n**Day 6: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach and Red Onion\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add chopped spinach and red onion. Scramble in a pan.\n*   **Dinner:** Lamb Rice Bowl with Tomato and Cucumber.\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced), Red Onion (thinly sliced).\n    *   *Instructions:* Combine rice, browned lamb, cucumber, tomato, and red onion.\n\n**Day 7: Sunday**\n\n*   **Breakfast:** Omelette with Tomato and Olives\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato, 1 tbsp Olives, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped tomato and olives. Cook as an omelette.\n*   **Dinner:** Mediterranean Lentil Soup (using remaining lentils)\n    *   *Ingredients:* Remaining Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), Remaining Chickpeas, Tomato (diced), water/broth, Lemon juice, Olive Oil.\n    *   *Instructions:* Sauté onion and garlic in olive oil. Add lentils, tomatoes, chickpeas, and water/broth. Simmer until lentils are tender. Season with lemon juice, salt, and pepper.\n\n**Rewarding Day : Day 8, Monday**\n*   **Breakfast:** Avocado and Egg Toast with Feta Cheese and Hot Sauce\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached), Hot Sauce.\n    *   *Instructions:* Toast bread, spread avocado, top with egg. Sprinkle hot sauce\n*   **Dinner:** Tuna Salad Sandwich\n    *   *Ingredients:* Tuna In water(0.33 can), Whole Wheat Bread(2), Spinach.\n    *   *Instructions:* Drain tuna, add some spinach. Put it on bread slices.\n\n## **Week 2: Chicken and Mediterranean Continued**\n\n**Day 8: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach.\n    *   *Ingredients:* 2 Eggs, handful of Spinach, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach. Scramble in a pan.\n*   **Dinner:** Chicken and Rice Bowl with Cucumber and Tomato\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced).\n    *   *Instructions:* Combine rice, chicken, cucumber, and tomato.\n\n**Day 9: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken Salad with Lettuce and Tomato\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and shredded), Lettuce, 1/4 Tomato (diced), Olive Oil, Lemon juice.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Dress with olive oil and lemon juice.\n\n**Day 10: Wednesday**\n\n*   **Breakfast:** Omelette with Spinach and Tomato\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Cook as an omelette.\n*   **Dinner:** Chicken and Chickpea Salad\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/4 can Chickpeas, Lettuce, Cucumber.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 11: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Red Onion\n    *   *Ingredients:* 2 Eggs, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add red onion. Scramble in a pan.\n*   **Dinner:** Chicken and Rice with Spinach\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, handful of Spinach.\n    *   *Instructions:* Combine rice, chicken, and spinach.\n\n**Day 12: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken and Vegetable Skewers (deconstructed)\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), Tomato, Cucumber, Red Onion.\n    *   *Instructions:* Arrange the ingredients on a plate.\n\n**Day 13: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Tomato\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato (diced).\n    *   *Instructions:* Whisk eggs, add tomato. Scramble in a pan.\n*   **Dinner:** Chicken Rice Bowl with Tomato and Cucumber.\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced).\n    *   *Instructions:* Combine rice, chicken, cucumber, and tomato.\n\n**Day 14: Sunday**\n\n*   **Breakfast:** Omelette with Spinach\n    *   *Ingredients:* 2 Eggs, handful of Spinach, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach. Cook as an omelette.\n*   **Dinner:** Chicken Salad Sandwich\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n\n## **Week 3: Tuna and Chicken Finale**\n\n**Day 15: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach and Red Onion\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add chopped spinach and red onion. Scramble in a pan.\n*   **Dinner:** Tuna and Quinoa Salad\n    *   *Ingredients:* 0.33 can Tuna, 1/2 cup Cooked Quinoa, Cucumber, Tomato, Red Onion.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 16: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken and Quinoa Bowl with Spinach\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Quinoa, handful of Spinach.\n    *   *Instructions:* Combine quinoa, chicken, and spinach.\n\n**Day 17: Wednesday**\n\n*   **Breakfast:** Omelette with Tomato and Feta\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato, 1 oz Feta Cheese.\n    *   *Instructions:* Whisk eggs, add chopped tomato and feta. Cook as an omelette.\n*   **Dinner:** Tuna Salad with Lettuce and Cucumber\n    *   *Ingredients:* 0.33 can Tuna, Lettuce, 1/4 Cucumber (diced), Olive Oil, Lemon juice.\n    *   *Instructions:* Combine tuna and cucumber. Dress with olive oil and lemon juice. Serve over lettuce.\n\n**Day 18: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Olives\n    *   *Ingredients:* 2 Eggs, 1 tbsp Olives (chopped).\n    *   *Instructions:* Whisk eggs, add olives. Scramble in a pan.\n*   **Dinner:** Chicken and Quinoa with Tomato and Cucumber\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Quinoa, Tomato, Cucumber.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 19: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Tuna and Chickpea Salad\n    *   *Ingredients:* 0.33 can Tuna, Remaining Chickpeas, Lettuce, Red Onion.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 20: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Feta Cheese\n    *   *Ingredients:* 2 Eggs, 1 oz Feta Cheese.\n    *   *Instructions:* Whisk eggs, add feta. Scramble in a pan.\n*   **Dinner:** Chicken Rice Bowl with Tomato and Cucumber and Chickpeas\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced). 0.33 can Chickpeas\n    *   *Instructions:* Combine rice, chicken, cucumber, tomato, Chickpeas\n\n**Day 21: Sunday**\n\n*   **Breakfast:** Omelette with Remaining Ingredients (Spinach, Tomato, etc.)\n    *   *Ingredients:* 2 Eggs, Remaining Spinach/Tomato/Onion (if any).\n    *   *Instructions:* Whisk eggs, add remaining vegetables. Cook as an omelette.\n*   **Dinner:** Chicken Salad Sandwich\n    *   *Ingredients:* Remaining Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n\n**Rewarding Day : Day 22, Monday**\n*   **Breakfast:** Chicken Salad Sandwich\n    *   *Ingredients:* Remaining Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n*   **Dinner:** Tuna Melt\n    *   *Ingredients:* 0.33 can Tuna, Remaining Feta Cheese, Whole Wheat Bread (2 slices).\n    *   *Instructions:* Combine tuna and feta. Place on bread and toast until cheese melts.\n\nEnjoy your culinary journey! Let me know if you have any other questions or need adjustments.\n", 'time': 'Sun Mar 23 12:00:39 2025', 'prompt': '\n        The recipes displayed are for a duration of 3 weeks. They include the following - Chicken,Fish,Eggs.\n        You will be provided recipes for : Breakfast,Dinner. No: of servings per meal : 1.\n        Your selected cuisine preferences are : Mediterranean,American. \n        comments you would like to add : \n        '}]

In [47]:
to_show[0]['time']

'Sun Mar 23 12:00:39 2025'

In [59]:
[ i for i in to_show if str(i['time'])=='Sun Mar 23 12:00:39 2025']

[{'_id': '67e063dcb7315b58ceabdcfb',
  'email': 'parab.aryan@gmail.com',
  'recipe': "Okay, Chef! Here's a 3-week Mediterranean/American fusion meal plan using your grocery list. I've focused on using all ingredients efficiently and providing variety while sticking to your specified meals (Breakfast and Dinner) and serving sizes (1).\n\n**Important Notes:**\n\n*   **Seasoning:** I'm assuming you have basic seasonings like salt, pepper, and maybe some dried herbs (oregano, basil, thyme). Adjust seasoning to your liking.\n*   **Flexibility:** Feel free to swap meals between days if something sounds better to you on a different day. This is a guide, not a rigid rulebook!\n*   **Storage:** Store leftovers properly in the refrigerator.\n*   **Ingredient Optimization:** I've planned to use ingredients across multiple weeks to minimize waste.\n\n## **Week 1: Mediterranean Start**\n\n**Day 1: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with diced Tomatoes and Spinach (from Ralphs).\n    

In [60]:
rr = [{'_id': 
       ('67e063dcb7315b58ceabdcfb'), 'email': 'parab.aryan@gmail.com', 'recipe': "Okay, Chef! Here's a 3-week Mediterranean/American fusion meal plan using your grocery list. I've focused on using all ingredients efficiently and providing variety while sticking to your specified meals (Breakfast and Dinner) and serving sizes (1).\n\n**Important Notes:**\n\n*   **Seasoning:** I'm assuming you have basic seasonings like salt, pepper, and maybe some dried herbs (oregano, basil, thyme). Adjust seasoning to your liking.\n*   **Flexibility:** Feel free to swap meals between days if something sounds better to you on a different day. This is a guide, not a rigid rulebook!\n*   **Storage:** Store leftovers properly in the refrigerator.\n*   **Ingredient Optimization:** I've planned to use ingredients across multiple weeks to minimize waste.\n\n## **Week 1: Mediterranean Start**\n\n**Day 1: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with diced Tomatoes and Spinach (from Ralphs).\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Scramble in a pan.\n*   **Dinner:** Mediterranean Lentil Soup\n    *   *Ingredients:* 1/4 cup Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), 1/4 can Chickpeas, 1/2 Tomato (diced), water/broth, Lemon juice, Olive Oil.\n    *   *Instructions:* Saute onion and garlic in olive oil. Add lentils, tomatoes, chickpeas, and water/broth. Simmer until lentils are tender. Season with lemon juice, salt, and pepper.\n\n**Day 2: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Lamb and Rice Bowl with Cucumber and Tomato Salad\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced), Red Onion (thinly sliced).\n    *   *Instructions:* Combine rice, browned lamb, cucumber, tomato, and red onion.\n\n**Day 3: Wednesday**\n\n*   **Breakfast:** Omelette with Spinach and Tomato\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Cook as an omelette.\n*   **Dinner:** Chickpea and Vegetable Salad with Lemon Vinaigrette\n    *   *Ingredients:* 1/4 can Chickpeas, Lettuce, 1/4 Tomato (diced), 1/4 Cucumber (diced), Red Onion (thinly sliced), Lemon juice, Olive Oil.\n    *   *Instructions:* Combine all ingredients. Dress with lemon juice and olive oil.\n\n**Day 4: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Olives and Red Onion\n    *   *Ingredients:* 2 Eggs, 1 tbsp Olives (chopped), 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add olives and red onion. Scramble in a pan.\n*   **Dinner:** Lamb and Lentil Stew\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/4 cup Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), water/broth, Tomato (diced).\n    *   *Instructions:* Saute onion and garlic in olive oil. Add lamb, lentils, tomatoes, and water/broth. Simmer until lentils are tender.\n\n**Day 5: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Greek Salad with Chickpeas\n    *   *Ingredients:* Lettuce, Tomato, Cucumber, Red Onion, 1/4 can Chickpeas, Olives, Lemon juice, Olive Oil.\n    *   *Instructions:* Combine all ingredients. Dress with lemon juice and olive oil.\n\n**Day 6: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach and Red Onion\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add chopped spinach and red onion. Scramble in a pan.\n*   **Dinner:** Lamb Rice Bowl with Tomato and Cucumber.\n    *   *Ingredients:* 1/6 lb Ground Lamb (browned), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced), Red Onion (thinly sliced).\n    *   *Instructions:* Combine rice, browned lamb, cucumber, tomato, and red onion.\n\n**Day 7: Sunday**\n\n*   **Breakfast:** Omelette with Tomato and Olives\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato, 1 tbsp Olives, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped tomato and olives. Cook as an omelette.\n*   **Dinner:** Mediterranean Lentil Soup (using remaining lentils)\n    *   *Ingredients:* Remaining Lentils, 1/4 Red Onion (diced), 1 clove Garlic (minced), Remaining Chickpeas, Tomato (diced), water/broth, Lemon juice, Olive Oil.\n    *   *Instructions:* Sauté onion and garlic in olive oil. Add lentils, tomatoes, chickpeas, and water/broth. Simmer until lentils are tender. Season with lemon juice, salt, and pepper.\n\n**Rewarding Day : Day 8, Monday**\n*   **Breakfast:** Avocado and Egg Toast with Feta Cheese and Hot Sauce\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached), Hot Sauce.\n    *   *Instructions:* Toast bread, spread avocado, top with egg. Sprinkle hot sauce\n*   **Dinner:** Tuna Salad Sandwich\n    *   *Ingredients:* Tuna In water(0.33 can), Whole Wheat Bread(2), Spinach.\n    *   *Instructions:* Drain tuna, add some spinach. Put it on bread slices.\n\n## **Week 2: Chicken and Mediterranean Continued**\n\n**Day 8: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach.\n    *   *Ingredients:* 2 Eggs, handful of Spinach, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach. Scramble in a pan.\n*   **Dinner:** Chicken and Rice Bowl with Cucumber and Tomato\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced).\n    *   *Instructions:* Combine rice, chicken, cucumber, and tomato.\n\n**Day 9: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken Salad with Lettuce and Tomato\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and shredded), Lettuce, 1/4 Tomato (diced), Olive Oil, Lemon juice.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Dress with olive oil and lemon juice.\n\n**Day 10: Wednesday**\n\n*   **Breakfast:** Omelette with Spinach and Tomato\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1/4 Tomato, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach and tomato. Cook as an omelette.\n*   **Dinner:** Chicken and Chickpea Salad\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/4 can Chickpeas, Lettuce, Cucumber.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 11: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Red Onion\n    *   *Ingredients:* 2 Eggs, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add red onion. Scramble in a pan.\n*   **Dinner:** Chicken and Rice with Spinach\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, handful of Spinach.\n    *   *Instructions:* Combine rice, chicken, and spinach.\n\n**Day 12: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken and Vegetable Skewers (deconstructed)\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), Tomato, Cucumber, Red Onion.\n    *   *Instructions:* Arrange the ingredients on a plate.\n\n**Day 13: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Tomato\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato (diced).\n    *   *Instructions:* Whisk eggs, add tomato. Scramble in a pan.\n*   **Dinner:** Chicken Rice Bowl with Tomato and Cucumber.\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced).\n    *   *Instructions:* Combine rice, chicken, cucumber, and tomato.\n\n**Day 14: Sunday**\n\n*   **Breakfast:** Omelette with Spinach\n    *   *Ingredients:* 2 Eggs, handful of Spinach, salt, pepper.\n    *   *Instructions:* Whisk eggs, add chopped spinach. Cook as an omelette.\n*   **Dinner:** Chicken Salad Sandwich\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n\n## **Week 3: Tuna and Chicken Finale**\n\n**Day 15: Monday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Spinach and Red Onion\n    *   *Ingredients:* 2 Eggs, handful of Spinach, 1 tbsp Red Onion (diced).\n    *   *Instructions:* Whisk eggs, add chopped spinach and red onion. Scramble in a pan.\n*   **Dinner:** Tuna and Quinoa Salad\n    *   *Ingredients:* 0.33 can Tuna, 1/2 cup Cooked Quinoa, Cucumber, Tomato, Red Onion.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 16: Tuesday**\n\n*   **Breakfast:** Avocado Toast with Egg\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Chicken and Quinoa Bowl with Spinach\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Quinoa, handful of Spinach.\n    *   *Instructions:* Combine quinoa, chicken, and spinach.\n\n**Day 17: Wednesday**\n\n*   **Breakfast:** Omelette with Tomato and Feta\n    *   *Ingredients:* 2 Eggs, 1/4 Tomato, 1 oz Feta Cheese.\n    *   *Instructions:* Whisk eggs, add chopped tomato and feta. Cook as an omelette.\n*   **Dinner:** Tuna Salad with Lettuce and Cucumber\n    *   *Ingredients:* 0.33 can Tuna, Lettuce, 1/4 Cucumber (diced), Olive Oil, Lemon juice.\n    *   *Instructions:* Combine tuna and cucumber. Dress with olive oil and lemon juice. Serve over lettuce.\n\n**Day 18: Thursday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Olives\n    *   *Ingredients:* 2 Eggs, 1 tbsp Olives (chopped).\n    *   *Instructions:* Whisk eggs, add olives. Scramble in a pan.\n*   **Dinner:** Chicken and Quinoa with Tomato and Cucumber\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Quinoa, Tomato, Cucumber.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 19: Friday**\n\n*   **Breakfast:** Avocado and Egg on Toast\n    *   *Ingredients:* 1 slice Whole Wheat Bread, 1/4 Avocado (mashed), 1 Egg (fried or poached).\n    *   *Instructions:* Toast bread, spread avocado, top with egg.\n*   **Dinner:** Tuna and Chickpea Salad\n    *   *Ingredients:* 0.33 can Tuna, Remaining Chickpeas, Lettuce, Red Onion.\n    *   *Instructions:* Combine all ingredients.\n\n**Day 20: Saturday**\n\n*   **Breakfast:** Scrambled Eggs (2) with Feta Cheese\n    *   *Ingredients:* 2 Eggs, 1 oz Feta Cheese.\n    *   *Instructions:* Whisk eggs, add feta. Scramble in a pan.\n*   **Dinner:** Chicken Rice Bowl with Tomato and Cucumber and Chickpeas\n    *   *Ingredients:* 1/4 lb Chicken Breast (cooked and diced), 1/2 cup Cooked Rice, 1/4 Cucumber (diced), 1/4 Tomato (diced). 0.33 can Chickpeas\n    *   *Instructions:* Combine rice, chicken, cucumber, tomato, Chickpeas\n\n**Day 21: Sunday**\n\n*   **Breakfast:** Omelette with Remaining Ingredients (Spinach, Tomato, etc.)\n    *   *Ingredients:* 2 Eggs, Remaining Spinach/Tomato/Onion (if any).\n    *   *Instructions:* Whisk eggs, add remaining vegetables. Cook as an omelette.\n*   **Dinner:** Chicken Salad Sandwich\n    *   *Ingredients:* Remaining Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n\n**Rewarding Day : Day 22, Monday**\n*   **Breakfast:** Chicken Salad Sandwich\n    *   *Ingredients:* Remaining Chicken Breast (cooked and shredded), Whole Wheat Bread (2 slices), Lettuce, Tomato.\n    *   *Instructions:* Combine chicken, lettuce, and tomato. Place on bread.\n*   **Dinner:** Tuna Melt\n    *   *Ingredients:* 0.33 can Tuna, Remaining Feta Cheese, Whole Wheat Bread (2 slices).\n    *   *Instructions:* Combine tuna and feta. Place on bread and toast until cheese melts.\n\nEnjoy your culinary journey! Let me know if you have any other questions or need adjustments.\n", 'time': 'Sun Mar 23 12:00:39 2025', 'prompt': '\n        The recipes displayed are for a duration of 3 weeks. They include the following - Chicken,Fish,Eggs.\n        You will be provided recipes for : Breakfast,Dinner. No: of servings per meal : 1.\n        Your selected cuisine preferences are : Mediterranean,American. \n        comments you would like to add : \n        '}]


In [ ]:
[i for i in rr if i['time']=="Sun Mar 23 2:00:39 2025"]

[]